### 1. Instalación de las Librerías


In [100]:
! pip install openai azure-search-documents python-dotenv pandas numpy



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


### 2. Cargar Variables de Entorno

PERO PRIMERO limpiamos las variables de entorno que se pudieron usar antes

In [103]:
# Limpia las variables para evitar conflictos
os.environ.pop("CHAT_MODEL_NAME", None)
os.environ.pop("EMBEDDING_MODEL_NAME", None)

# Carga el archivo .env
load_dotenv(find_dotenv())

# Variables necesarias
CHAT_MODEL = os.getenv("CHAT_MODEL_NAME")
EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL_NAME")

print(f"Modelo de chat cargado: {CHAT_MODEL}")
print(f"Modelo de embeddings cargado: {EMBEDDING_MODEL}")


Modelo de chat cargado: gpt-4
Modelo de embeddings cargado: text-embedding-ada-002


In [105]:
from dotenv import load_dotenv, find_dotenv
import os

# Cargar las variables de entorno
load_dotenv(find_dotenv())

# Variables necesarias
API_KEY = os.getenv("OPENAI_API_KEY")
RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE", "").strip()
API_TYPE = os.getenv("OPENAI_API_TYPE", "azure")
API_VERSION = os.getenv("OPENAI_API_VERSION", "2023-05-15")
CHAT_MODEL = os.getenv("CHAT_MODEL_NAME")
EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL_NAME")
JSON_FILES_PATH = os.getenv("JSON_FILES_PATH")

# Verificar que el archivo existe
if not os.path.isfile(JSON_FILES_PATH):
    raise FileNotFoundError(f"El archivo {JSON_FILES_PATH} no existe.")

print(f"Configuración cargada:\n- Archivo JSON: {JSON_FILES_PATH}\n- Modelo de chat: {CHAT_MODEL}\n- Modelo de embeddings: {EMBEDDING_MODEL}")


Configuración cargada:
- Archivo JSON: /Users/luisalbertocerelli/Desktop/00-Todo/14_Hackaton_Real/PRINCIPAL/01Conversion_File_Json/Json_conversion/documentos_indexados.json
- Modelo de chat: gpt-4
- Modelo de embeddings: text-embedding-ada-002


### 3. Configuración de OpenAI con Azure


In [106]:
import openai

# Configuracion de OpenAI:

openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = "2023-06-01-preview"

print("Configuración de OpenAI completada.")

Configuración de OpenAI completada.


### 4. Configuración de Azure Cognitive Search

Es el servicio para almacenar y buscar documentos relevantes. Aquí configuramos:

* El endpoint de tu servicio de búsqueda.
* La clave API para autenticación.
* El cliente para interactuar con el índice.

In [107]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv, find_dotenv
import os

# Cargar las variables de entorno
load_dotenv(find_dotenv())

# Configuración de Azure Cognitive Search desde el .env
search_service_endpoint = os.getenv("SEARCH_SERVICE_ENDPOINT")
index_name = os.getenv("SEARCH_INDEX_NAME")
search_api_key = os.getenv("SEARCH_API_KEY")

# Validar que las variables necesarias están configuradas
if not search_service_endpoint or not index_name or not search_api_key:
    raise ValueError("Faltan valores en el archivo .env para configurar Azure Cognitive Search.")

# Crear el cliente de búsqueda
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_api_key)
)

print("Cliente de búsqueda configurado correctamente.")


Cliente de búsqueda configurado correctamente.


### 5. Función para Cargar Documentos en Azure Cognitive Search

In [19]:
import os
import json
import base64
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchableField, SimpleField, SearchIndex
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv
import re

# Cargar variables del archivo .env
load_dotenv()
search_service_endpoint = os.getenv("SEARCH_SERVICE_ENDPOINT")
search_api_key = os.getenv("SEARCH_API_KEY")
index_name = os.getenv("INDEX_NAME")
json_files_path = os.getenv("JSON_FILES_PATH")

def normalizar_clave(clave):
    """
    Normaliza la clave para cumplir con los requisitos de Azure Search.
    Convierte caracteres no válidos en guiones bajos (_).
    """
    # Reemplazar caracteres no permitidos con guión bajo
    return re.sub(r"[^a-zA-Z0-9_\-]", "_", clave)

def cargar_documentos_en_search():
    # Validar variables esenciales
    if not search_service_endpoint or not search_api_key or not index_name:
        print("Error: Asegúrate de que las variables SEARCH_SERVICE_ENDPOINT, SEARCH_API_KEY y INDEX_NAME están definidas en el archivo .env")
        return

    # Crear cliente para el índice
    index_client = SearchIndexClient(
        endpoint=search_service_endpoint,
        credential=AzureKeyCredential(search_api_key)
    )

    # Definir el esquema del índice
    fields = [
        SimpleField(name="id", type="Edm.String", key=True),
        SearchableField(name="content", type="Edm.String", analyzer_name="en.microsoft"),
    ]
    index = SearchIndex(name=index_name, fields=fields)

    # Crear el índice si no existe
    try:
        if index_name not in [i.name for i in index_client.list_indexes()]:
            index_client.create_index(index)
            print(f"Índice '{index_name}' creado exitosamente.")
        else:
            print(f"Índice '{index_name}' ya existe. Continuando...")
    except Exception as e:
        print(f"Error al gestionar el índice: {e}")
        return

    # Cargar documentos desde el archivo JSON
    try:
        with open(json_files_path, "r", encoding="utf-8") as json_file:
            raw_documents = json.load(json_file)
            # Normalizar las claves de los documentos
            documents = [
                {"id": normalizar_clave(doc["id"]), "content": doc["content"]}
                for doc in raw_documents
                if "id" in doc and "content" in doc
            ]
    except Exception as e:
        print(f"Error al leer el archivo JSON: {e}")
        return

    # Crear cliente para interactuar con el índice
    search_client = SearchClient(
        endpoint=search_service_endpoint,
        index_name=index_name,
        credential=AzureKeyCredential(search_api_key)
    )

    # Subir documentos al índice
    try:
        upload_response = search_client.upload_documents(documents=documents)
        print(f"Documentos cargados exitosamente: {upload_response}")
    except Exception as e:
        print(f"Error al cargar documentos en el índice: {e}")

# Llamar a la función
if __name__ == "__main__":
    cargar_documentos_en_search()


Índice 'documentos-huella' ya existe. Continuando...
Documentos cargados exitosamente: [<azure.search.documents._generated.models._models_py3.IndexingResult object at 0x160693da0>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x1606aa2d0>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x1606a9880>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x1606aaf00>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x1606aad50>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x1606aae40>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x1606aacc0>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x1606aad20>, <azure.search.documents._generated.models._models_py3.IndexingResult object at 0x1606aae70>, <azure.search.documents._generated.models._models_py3.IndexingResult object

### 6. Búsqueda de Documentos Relevantes (solo para probar si esta funcionando bien)

In [108]:
import os
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

# Configuración desde las variables del entorno
service_endpoint = os.environ["SEARCH_SERVICE_ENDPOINT"]
api_key = os.environ["SEARCH_API_KEY"]
index_name = os.environ["INDEX_NAME"]

# Crear cliente de búsqueda
search_client = SearchClient(
    endpoint=service_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(api_key)
)

# Función para buscar documentos
def buscar_documentos(query, top=5):
    results = search_client.search(
        search_text=query,
        top=top,
        query_type="simple"
    )
    documentos = [doc['content'] for doc in results]
    return documentos

# Ejemplo de uso
if __name__ == "__main__":
    query = "huella de carbono"
    resultados = buscar_documentos(query, top=3)
    print("Resultados de la búsqueda:")
    for i, doc in enumerate(resultados, start=1):
        print(f"{i}. {doc}")



Resultados de la búsqueda:
1. Hola y bienvenidos al Cuestionario de Gestión de proyectos de Huella 
de Carbono 
Gracias por leer, completar los datos requeridos y enviarlo
Hemos diseñado este cuestionario para poder comprender mejor la situación de su entidad en términos
de sostenibilidad, su disposición a embarcarse en un plan de gestión de huella de carbono con el soporte
de la Asociación Huella de Carbono y así poder ayudarle mejor y de forma más personalizada.
Su participación en este cuestionario es esencial para obtener una visión completa y precisa de la
información que necesitamos para poder ayudarle.
Este cuestionario se ha diseñado para ser claro y fácil de seguir.
Los campos con un asterisco en Rojo * son obligatorios
Las preguntas abiertas están destinadas a recoger sus valiosas opiniones y experiencias, mientras que
las preguntas cerradas nos proporcionarán datos cuantitativos que serán fundamentales para nuestro
análisis.
Las respuestas serán utilizadas para preparar el p